# Preprocessing and data set creation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from astropy.io import fits
from TNGDataSet import TNGDataSet
plt.style.use(astropy_mpl_style)

In [ ]:
# Dummy scaling function
def ScaleImage(img):
    return img[0:150,0:150].astype('float32')

## Tracing function
set to 0 for no traces up to 4 for alot of traces

In [ ]:
os.environ["DIFF_TRACE"]="0"

## Set up dataset creation arguments and load the data set

Dataset is only loaded the first time and will be stored locally in data_dir/tng_data_set folder


In [2]:
data_dir='/scratch/astroinfo2023/diffusion/'

If you wish to reload you need to delete all local datasets

In [ ]:
#!rm -rfv '/scratch/astroinfo2023/diffusion/tng_data_set/'

In [ ]:
# This is useless :)
dataset_version = '1.0.0'

In [ ]:
# Training set preparation
def load_dataset(batch_size):

    arg_dict = {
    #Increment version to reload DS
    'train_percent':0.0001,
    'val_percent':0.0001, 
    'test_percent':0.0001 , 
    'generation_verbosity' : 10 , 
    'Scaler_fcn':ScaleImage,
    # Image size variable does not work .. it is always (150,150)
    'Image_Size' : (150,150),
   #'band_filters' : ['CFHT_MEGACAM.U', 'SUBARU_HSC.G' ,'SUBARU_HSC.R' ,'CFHT_MEGACAM.R' ,'SUBARU_HSC.I', 'SUBARU_HSC.Z' ,'SUBARU_HSC.Y']
    'band_filters' : ['CFHT_MEGACAM.R']
    }
    #subsets = [tfds.Split.TRAIN,tfds.Split.VALIDATION, tfds.Split.TEST]
    subsets = [tfds.Split.TRAIN]
    ds = tfds.load('TNGDataSet',
                   split=subsets,
                   data_dir=data_dir,
                   builder_kwargs=arg_dict, 
                   shuffle_files=True)
    tng_ds = ds[0]
    tng_ds = tng_ds.shuffle(buffer_size=10*batch_size)
    tng_ds = tng_ds.repeat()
    tng_ds = tng_ds.batch(batch_size)
    tng_ds = tng_ds.prefetch(tf.data.experimental.AUTOTUNE)
    return tng_ds.as_numpy_iterator()
     



To recover the subsets you can recover them directly from the newly created dataset

In [ ]:
ds_tng= load_dataset(32)
batch = next(ds_tng)

# WIP : Plotting proto

In [ ]:
import random
for i in range(4):
    plt.figure()
    plt.subplot(131)
    plt.imshow(batch['img'][random.randint(0, 32)])